In [1]:
from darts import TimeSeries
from darts.models import NBEATSModel, NHiTSModel, TransformerModel, TSMixerModel
from darts.utils.losses import *
from darts.metrics import metrics as darts_metrics
from utils import data_handling, helpers
import torch
import numpy as np
import os
import pickle
import pandas as pd
import config
import copy
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping


# Constants
DEVICE = [1]
IN_LEN = 96
OUT_LEN = 96
LOSS_FN = torch.nn.MSELoss()
LAYER_WIDTH = 256
NUM_STACKS = 4
NUM_BLOCKS = 2
NUM_LAYERS = 2
COEFFS_DIM = 5
DROPOUT = 0.25
VERBOSE = True
TRAIN_EPOCHS = 15
TUNE_EPOCHS = 5
four_weeks = -24*7*4
LR = 0.005

metrics_output_path = config.CONFIG_OUTPUT_PATH["darts"] / "darts_metrics.csv"

model_path = config.CONFIG_MODEL_LOCATION["darts"]

/vol/fob-vol7/nebenf21/reinbene/bene/MA/myenv/lib/python3.10/site-packages/statsforecast/core.py:26: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
def extend_source_to_target_id_count(source, target):
    source_id_count = source["train"].shape[1]
    target_id_count = target["train"].shape[1]

    full_repeats = target_id_count // source_id_count
    remainder = target_id_count % source_id_count

    repeated_tensor = source["train"].repeat(1, full_repeats)
    remainder_tensor = source["train"][:, :remainder]
    source_train = torch.cat((repeated_tensor, remainder_tensor), dim=1)
    
    assert target_id_count == source_train.size(1), f"Reshaping was incorrect. Target_train = {target_id_count}, source_train = {source_train.size(1)}."

    repeated_tensor = source["validation"].repeat(1, full_repeats)
    remainder_tensor = source["validation"][:, :remainder]
    source_validation = torch.cat((repeated_tensor, remainder_tensor), dim=1)
    assert target_id_count == source_validation.size(1), f"Reshaping was incorrect. Target_val = {target_id_count}, source_val = {source_validation.size(1)}."

    return source_train, source_validation


def process_tl_data(source_data, target_data):
    # either reshape source or target dataset according to which has less IDs
    source_ids = source_data["train"].size(1)
    target_ids = target_data["test"].size(1)

    fine_tune_horizon = -24*7*4
    target_test = target_data["test"]
    target_fine_tuning = target_data["train"][fine_tune_horizon:,:]

    # remove IDs if source is bigger than target or
    # repeat IDs if target is bigger than source
    if target_ids < source_ids:
        source_train = source_data["train"][:,:target_ids]
        source_validation = source_data["validation"][:,:target_ids]
    else:
        source_train, source_validation = extend_source_to_target_id_count(source_data, target_data)

    # convert to TimeSeries dataframe
    source_train = TimeSeries.from_values(source_train)
    source_validation = TimeSeries.from_values(source_validation)
    target_test = TimeSeries.from_values(target_test)
    target_fine_tuning = TimeSeries.from_values(target_fine_tuning)
    target_train = TimeSeries.from_values(target_data["train"])
    target_validation = TimeSeries.from_values(target_data["validation"])

    tl_dataset = {
                    "source_train" : source_train,
                    "source_validation" : source_validation,
                    "target_fine_tuning" : target_fine_tuning,
                    "target_test" : target_test,
                    "target_train" : target_train,
                    "target_validation" : target_validation
                }

    return tl_dataset

In [3]:
def train_nhits(ts_train, ts_val, epochs=1):   
    TRAINER_ARGS = {"enable_progress_bar": True, 
                "accelerator": "gpu",  
                "devices" : DEVICE,
             }

    nhits_model = NHiTSModel(
        input_chunk_length=IN_LEN,
        output_chunk_length=OUT_LEN,
        activation='ReLU',
        num_stacks=NUM_STACKS,
        num_blocks=NUM_BLOCKS,
        num_layers=NUM_LAYERS,
        layer_widths=LAYER_WIDTH,
        dropout=DROPOUT,
        loss_fn=LOSS_FN,
        use_reversible_instance_norm=True,
        optimizer_kwargs={"lr": LR},
        pl_trainer_kwargs=TRAINER_ARGS,
    )

    nhits_model.fit(ts_train, val_series=ts_val, epochs=epochs, verbose=VERBOSE)
    return nhits_model

# Train NBEATS model
def train_nbeats(ts_train, ts_val, epochs=1):   
    TRAINER_ARGS = {"enable_progress_bar": True, 
                "accelerator": "gpu",  
                "devices" : DEVICE,
             }

    nbeats_model = NBEATSModel(
        input_chunk_length=IN_LEN,
        output_chunk_length=OUT_LEN,
        batch_size=32,
        num_stacks=NUM_STACKS,
        num_blocks=NUM_BLOCKS,
        num_layers=NUM_LAYERS,
        layer_widths=LAYER_WIDTH,
        expansion_coefficient_dim=COEFFS_DIM,
        loss_fn=LOSS_FN,
        use_reversible_instance_norm=True,
        activation='ReLU',
        optimizer_kwargs={"lr": LR},
        pl_trainer_kwargs=TRAINER_ARGS,
    )    

    nbeats_model.fit(ts_train, val_series=ts_val, epochs=epochs, verbose=VERBOSE)
    return nbeats_model

# Train Transformer model
def train_transformer(ts_train, ts_val, epochs=1):    
    TRAINER_ARGS = {"enable_progress_bar": True, 
                "accelerator": "gpu",  
                "devices" : DEVICE,
             }

    transformer_model = TransformerModel(
        input_chunk_length=IN_LEN, 
        output_chunk_length=OUT_LEN,
        d_model=LAYER_WIDTH, 
        nhead=4, 
        num_encoder_layers=3, 
        num_decoder_layers=3, 
        dim_feedforward=LAYER_WIDTH, 
        dropout=DROPOUT, 
        activation='relu', 
        loss_fn=LOSS_FN,
        optimizer_kwargs={"lr": LR},
        use_reversible_instance_norm=True,
        pl_trainer_kwargs=TRAINER_ARGS,
        )
    
    transformer_model.fit(ts_train, val_series=ts_val, epochs=epochs, verbose=VERBOSE)
    return transformer_model 

# Train TSMixer model
def train_tsmixer(ts_train, ts_val, epochs=1):    
    TRAINER_ARGS = {"enable_progress_bar": True, 
                "accelerator": "gpu",  
                "devices" : DEVICE,
             }
    
    tsmixer_model = TSMixerModel(
        input_chunk_length=IN_LEN, 
        output_chunk_length=OUT_LEN, 
        hidden_size=LAYER_WIDTH, 
        ff_size=LAYER_WIDTH, 
        num_blocks=NUM_BLOCKS, 
        activation='ReLU', 
        dropout=DROPOUT, 
        loss_fn=LOSS_FN,
        norm_type='LayerNorm', 
        optimizer_kwargs={"lr": LR},
        use_reversible_instance_norm=True,
        pl_trainer_kwargs=TRAINER_ARGS,
    )

    tsmixer_model.fit(ts_train, val_series=ts_val, epochs=epochs, verbose=VERBOSE)
    return tsmixer_model

In [4]:
def evaluate(model, target_test):
    """
    Evaluates models on target test set
    Input:  -trained model
            -List of target test sets shaped according to models

    Output: Dict{MSE, MAE}
    """

    forecasting_endpoint = int(len(target_test)) - 96*2
    window = [target_test[i:i+96] for i in range(0, forecasting_endpoint, 5)]
    target = [target_test[i+96:i+96+96] for i in range(0, forecasting_endpoint, 5)]

    # predict over dataloader with slidingwindow implementation and 5 time step shifts for each input
    predictions = model.predict(n=96, series=window)

    mse = darts_metrics.mse(predictions, target)
    mae = darts_metrics.mae(predictions, target)

    mse = sum(mse) / len(predictions)
    mae = sum(mae) / len(predictions)

    return {'MSE': mse, 'MAE': mae}


def fine_tune_model(model, target_fine_tuning, epochs=1):
    """
    Fine tune models over specified epochs

    Input:  -trained models
            -fine tuning dataset
            -epochs

    Returns: fitted models
    """

    
    model.fit(
            target_fine_tuning,
            num_loader_workers=4,
            epochs=epochs,
            max_samples_per_ts=None,
        )
 
    return model

In [5]:
# use electricity dataset
electricity_dict = data_handling.format_electricity()

for key, value in electricity_dict.items():
			electricity_dict[key]= data_handling.df_to_tensor(value)

# normalize train and use matrics for val and test
electricity_dict["4_weeks_train"] = electricity_dict["train"][four_weeks:,:]
electricity_dict["train"], train_standardize_dict = helpers.custom_standardizer(electricity_dict["train"])
electricity_dict["validation"], _ = helpers.custom_standardizer(electricity_dict["validation"], train_standardize_dict)
electricity_dict["test"], _ = helpers.custom_standardizer(electricity_dict["test"], train_standardize_dict)
electricity_dict["4_weeks_train"], _ = helpers.custom_standardizer(electricity_dict["4_weeks_train"], train_standardize_dict)

# bavaria dataset
data_tensor = data_handling.load_bavaria_electricity()
bavaria_dict, standadizer = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)
bavaria_dict["4_weeks_train"] = bavaria_dict["train"][four_weeks:,:]

# building genome project dataset
data_tensor = data_handling.load_genome_project_data()
gp_dict, standadizer = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)
gp_dict["4_weeks_train"] = gp_dict["train"][four_weeks:,:]

Length train set: 209 days, 0:00:00
Length validation set: 34 days, 0:00:00
Saving train, validation and test df for faster loading


In [8]:
tl_setups = {
    "ELD_to_Bavaria" : (electricity_dict, bavaria_dict), 
    "ELD_to_GP2" : (electricity_dict, gp_dict),
    "Bavaria_to_ELD" : (bavaria_dict, electricity_dict), 
    "Bavaria_to_GP2" : (bavaria_dict, gp_dict), 
    "GP2_to_Bavaria": (gp_dict, bavaria_dict), 
    "GP2_to_ELD" : (gp_dict, electricity_dict)
     }

In [13]:
models = {
    "NHiTS": train_nhits,
  #  "NBEATS": train_nbeats,
    "Transformer": train_transformer,
    "TSMixer": train_tsmixer
}

# Initialize the DataFrame
try:
    results_df = pd.read_csv(metrics_output_path, index_col=[0, 1, 2])
except FileNotFoundError:
    metrics = ["MSE", "MAE"]
    learning_scenarios = ["Zero-Shot", "four_weeks_tl", "full_tl", "full_training", "four_weeks_training"]
    index = pd.MultiIndex.from_product([tl_setups.keys(), learning_scenarios, metrics], names=["Setup", "Learning_scenario", "Metric"])
    results_df = pd.DataFrame(columns=models.keys(), index=index)

# Helper functions
def update_metrics(setup_name, model_name, learning_scenario, mae, mse):
    results_df.loc[(setup_name, learning_scenario, "MAE"), model_name] = mae
    results_df.loc[(setup_name, learning_scenario, "MSE"), model_name] = mse

def is_metric_filled(setup_name, model_name, learning_scenario):
    # Check if specific metrics for a model in a setup and fine-tuning scenario are NaN or not
    metrics_filled = not results_df.loc[(setup_name, learning_scenario, slice(None)), model_name].isnull().any()
    return metrics_filled

def run_model_tl(setup_name, tl_data):
    source_train = tl_data["source_train"]
    source_val = tl_data["source_validation"]
    target_fine_tuning = tl_data["target_fine_tuning"]
    target_test = tl_data["target_test"]
    target_train = tl_data["target_train"]
    #target_val = tl_data["target_validation"]

    for model_name, model_func in models.items():
        if is_metric_filled(setup_name, model_name, "full_tl"):
            print(f"Skipping {model_name} for {setup_name} with full fine-tuning as metrics are already filled.")
            continue
        
        # Train model with source dataset (Zero-Shot)
        model = model_func(source_train, source_val, epochs=TRAIN_EPOCHS)
        metrics = evaluate(model, target_test)

        # Create deep copy for full fine-tuning
        full_tl_model = copy.deepcopy(model)

        update_metrics(setup_name, model_name, "Zero-Shot", metrics['MAE'], metrics['MSE'])
        print(f"Metrics updated for {model_name} in {setup_name} with zero-shot fine-tuning: MAE = {metrics['MAE']}, MSE = {metrics['MSE']}")
        
        # Fine-tune on small target train set (four_weeks_tl)
        model = fine_tune_model(model, target_fine_tuning, epochs=TUNE_EPOCHS + 5)
        metrics = evaluate(model, target_test)
        update_metrics(setup_name, model_name, "four_weeks_tl", metrics['MAE'], metrics['MSE'])

        # Fine-tune on full target train set (full_tl)
        full_tl_model = fine_tune_model(full_tl_model, target_train, epochs=TUNE_EPOCHS)
        metrics = evaluate(full_tl_model, target_test)
        update_metrics(setup_name, model_name, "full_tl", metrics['MAE'], metrics['MSE'])

        # Save after every dataset combination
        results_df.to_csv(metrics_output_path)

def train_baselines(setup_name, tl_data):
    target_fine_tuning = tl_data["target_fine_tuning"]
    target_test = tl_data["target_test"]
    target_train = tl_data["target_train"]
    target_validation = tl_data["target_validation"]

    for model_name, model_func in models.items():
        if is_metric_filled(setup_name, model_name, "four_weeks_training"):
            print(f"Skipping {model_name} for {setup_name} with as metrics are already filled.")
            continue

        # Train on full target train set (full_training)
        model = model_func(target_train, target_validation, epochs=TRAIN_EPOCHS)
        metrics = evaluate(model, target_test)
        update_metrics(setup_name, model_name, "full_training", metrics['MAE'], metrics['MSE'])

        # Train on short target train set (four_weeks_training)
        model = model_func(target_fine_tuning, target_validation, epochs=TRAIN_EPOCHS)
        metrics = evaluate(model, target_test)
        update_metrics(setup_name, model_name, "four_weeks_training", metrics['MAE'], metrics['MSE'])

        results_df.to_csv(metrics_output_path)

# Execute for each setup and fine-tuning scenario
for setup_name, (source_data, target_data) in tl_setups.items():
    print(setup_name)
    tl_data = process_tl_data(source_data, target_data)
    run_model_tl(setup_name, tl_data)
    train_baselines(setup_name, tl_data)

results_df

ELD_to_GP2
Skipping Transformer for ELD_to_GP2 with full fine-tuning as metrics are already filled.


/vol/fob-vol7/nebenf21/reinbene/bene/MA/myenv/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /vol/fob-vol7/nebenf21/reinbene/bene/MA/myenv/lib/py ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]

  | Name                | Type                | Params
------------------------------------------------------------
0 | criterion           | MSELoss             | 0     
1 | train_metrics       | MetricCollection    | 0     
2 | val_metrics         | MetricCollection    | 0     
3 | rin                 | RINorm              | 2.9 K 
4 | encoder             | Linear              | 372 K 
5 | positional_encoding | _PositionalEncoding |

Epoch 14: 100%|██████████| 314/314 [00:28<00:00, 11.17it/s, train_loss=0.777, val_loss=nan.0]

`Trainer.fit` stopped: `max_epochs=15` reached.


Epoch 14: 100%|██████████| 314/314 [00:28<00:00, 11.17it/s, train_loss=0.777, val_loss=nan.0]


...

Epoch 14: 100%|██████████| 16/16 [00:02<00:00,  5.41it/s, train_loss=0.766, val_loss=0.787]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Predicting DataLoader 0: 100%|██████████| 18/18 [00:01<00:00, 11.96it/s]
{'MSE': 1.0486344807326684, 'MAE': 0.666307353014484}


/tmp/ipykernel_25723/1685300477.py:19: PerformanceWarning: indexing past lexsort depth may impact performance.
  results_df.loc[(setup_name, learning_scenario, "MAE"), model_name] = mae
/tmp/ipykernel_25723/1685300477.py:20: PerformanceWarning: indexing past lexsort depth may impact performance.
  results_df.loc[(setup_name, learning_scenario, "MSE"), model_name] = mse


NHiTS  Transformer  \
Setup          Learning_scenario   Metric                              
ELD_to_Bavaria Zero-Shot           MSE     1.619146e-03     0.001618   
                                   MAE     2.976792e-02     0.029754   
               four_weeks_tl       MSE     4.697010e-04     0.000492   
                                   MAE     1.075261e-02     0.010888   
               full_tl             MSE     2.851176e-04     0.000328   
                                   MAE     9.179015e-03     0.009856   
               full_training       MSE     2.838999e-04     0.000315   
                                   MAE     8.886287e-03     0.009180   
               four_weeks_training MSE     3.550585e-04     0.000502   
                                   MAE     1.074552e-02     0.011654   
ELD_to_GP2     Zero-Shot           MSE     9.277600e-01     0.988170   
                                   MAE     6.164209e-01     0.656070   
               four_weeks_tl       MSE     9.950743e-01     1.019768   
                                   MAE     6.413549e-01     0.660914   
               full_tl             MSE     9.805492e-01     0.975071   
                                   MAE     6.532800e-01     0.647807   
               full_training       MSE     9.808304e-01          NaN   
                                   MAE     6.499664e-01          NaN   
               four_weeks_training MSE     3.812819e+00     1.048634   
                                   MAE     1.286878e+00     0.666307   
Bavaria_to_ELD Zero-Shot           MSE     1.430658e+01    10.621312   
                                   MAE     3.264933e+00     2.885642   
               four_weeks_tl       MSE     6.867430e-01     0.959930   
                                   MAE     6.430946e-01     0.813232   
               full_tl             MSE     3.637200e-01     0.947210   
                                   MAE     4.147567e-01     0.808654   
               full_training       MSE     3.588576e-01     0.947477   
                                   MAE     4.149607e-01     0.809668   
               four_weeks_training MSE     4.340198e-01     0.953537   
                                   MAE     4.742046e-01     0.812253   
Bavaria_to_GP2 Zero-Shot           MSE     1.035199e+06     7.735226   
                                   MAE     6.715389e+02     2.066784   
               four_weeks_tl       MSE     3.555567e+02     1.073695   
                                   MAE     1.141571e+01     0.679530   
               full_tl             MSE     9.348713e-01     0.975385   
                                   MAE     6.251944e-01     0.648239   
               full_training       MSE     9.746067e-01     0.975527   
                                   MAE     6.488926e-01     0.648849   
               four_weeks_training MSE     1.160768e+00     1.045447   
                                   MAE     6.961357e-01     0.664533   
GP2_to_Bavaria Zero-Shot           MSE     1.653185e-03     0.001701   
                                   MAE     3.008492e-02     0.030630   
               four_weeks_tl       MSE     3.885315e-04     0.000488   
                                   MAE     1.029623e-02     0.010714   
               full_tl             MSE     2.770908e-04     0.000326   
                                   MAE     8.923608e-03     0.009737   
               full_training       MSE     2.899523e-04     0.000322   
                                   MAE     9.143972e-03     0.009527   
               four_weeks_training MSE     3.392494e-04     0.000499   
                                   MAE     9.836575e-03     0.011501   
GP2_to_ELD     Zero-Shot           MSE     9.585188e-01     0.959904   
                                   MAE     8.120929e-01     0.811470   
               four_weeks_tl       MSE     4.855094e-01     0.951312   
                                   MAE     5.192017e-01     0.812263   
               ful